In [3]:
# 六派神見選股法

In [3]:
import json
import requests
import pandas as pd
import sys


In [45]:
url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20200617&stockNo=2330&_=1592394271327"

res = requests.get(url)
json_stock = json.loads(res.content)

In [44]:
for i in range(len(json_stock["data"])):
    print(json_stock["data"][i][0])
    print(json_stock["data"][i][1])
    print("--------------")

109/06/01
37,936,214
--------------
109/06/02
26,663,587
--------------
109/06/03
67,894,337
--------------
109/06/04
47,225,322
--------------
109/06/05
44,077,262
--------------
109/06/08
52,042,921
--------------
109/06/09
37,068,082
--------------
109/06/10
42,207,259
--------------
109/06/11
50,612,255
--------------
109/06/12
46,970,918
--------------
109/06/15
51,032,404
--------------
109/06/16
40,725,216
--------------
109/06/17
34,692,968
--------------


In [46]:
for i in range(1000):
    res = requests.get(url)
    print("hppt status: ",res.status_code , "現在count數:",i)
    

hppt status:  200 現在count數: 0
hppt status:  200 現在count數: 1
hppt status:  200 現在count數: 2
hppt status:  200 現在count數: 3
hppt status:  200 現在count數: 4
hppt status:  200 現在count數: 5
hppt status:  200 現在count數: 6
hppt status:  200 現在count數: 7
hppt status:  200 現在count數: 8
hppt status:  200 現在count數: 9
hppt status:  200 現在count數: 10
hppt status:  200 現在count數: 11
hppt status:  200 現在count數: 12
hppt status:  200 現在count數: 13
hppt status:  200 現在count數: 14
hppt status:  200 現在count數: 15
hppt status:  200 現在count數: 16
hppt status:  200 現在count數: 17
hppt status:  200 現在count數: 18
hppt status:  200 現在count數: 19
hppt status:  200 現在count數: 20
hppt status:  200 現在count數: 21
hppt status:  200 現在count數: 22
hppt status:  200 現在count數: 23
hppt status:  200 現在count數: 24
hppt status:  200 現在count數: 25


ConnectionError: HTTPSConnectionPool(host='www.twse.com.tw', port=443): Max retries exceeded with url: /exchangeReport/STOCK_DAY?response=json&date=20200617&stockNo=2330&_=1592394271327 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000186C47F0860>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。',))